# Stack Overflow

## Introduction 

In the second part of this assignment, we will create and analyze time series of creation dates of Stack Overflow questions. This assignment is to be completed **INDIVIDUALLY** and it is due on **February 23 at 3pm**.

In [ ]:
print("Aaron Elliot")
print("BU ID: U71617701")

Let's create some time series from the data. You may choose to analyze either users or tags. To analyze users, take the top 100 users with the most question posts. For each user, your time series will be the number of questions posted by that user at some frequency. To analyze tags, take the top 100 most popular question tags. For each tag, your time series will be the number of questions with that tag at some frequency. You may choose to sample your data each week, each month, on a certain day of the week or at certain hours in a day depending on what trend you are hoping to find in the data. For example, if you choose to analyze tags and sample during different hours of the day, your hypothesis could be that languages (e.g. Javascript) that are used more in industry will have more questions posted during work hours, whereas languages (e.g. Python) that are taught in academia will have more questions posted after midnight when students are scrambling to finish their homework.

Compare the time series using one of the methods discussed in class. In a few paragraphs, write down what you were hoping to find in the data, what timeseries you created, what method you chose and why. **(25 pts)**

Plot the 2 most similar timeseries in one graph, and plot the 2 most different time series in another graph. Label your axes. **(5 pts)**

You may find the [pandas.DataFrame.resample](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html) module helpful.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import sys
import math
from lxml import etree

xmlfile = 'stackoverflow-posts-2015.xml'
context = etree.iterparse(xmlfile)
i=0
TagDic = {}   #Dictionary of the Tags
Top100 = []      #List of the Top 100 Tags
TSDic = {} #Dictionary of all the timeseries, resampled daily by

def EucDistance (vec1, vec2):
    dist = 0
    for x in range(0,len(vec1)):
        dist += (vec1[x]-vec2[x])**2
    return math.sqrt(dist)

def L1Distance (vec1, vec2):
    dist = 0
    for x in range(0,len(vec1)):
        dist += math.fabs(vec1[x]-vec2[x])
    return dist

def tryPartition (StringOrNone):
    if StringOrNone == None:
        return 'none'
    else:
        return StringOrNone.partition('>')[0].partition('<')[2]
    
for action, elem in context: #for values in the xml, we extract the CreationDate and the First Tag
    if elem.get('PostTypeId') == "1": # if post is an question
        Tag = tryPartition(elem.get('Tags'))
        if Tag == 'none':
            continue
        if not(Tag in TagDic):
            TagDic[Tag] = {'tag':Tag, 'dates':[pd.to_datetime(elem.get('CreationDate'))]}
        else:
            TagDic[Tag]['dates'].append(pd.to_datetime(elem.get('CreationDate')))
    if i>100000:
        break
        i+=1
        
for Tag in TagDic: # This sorts out the top 100 Tags, and puts them in Top100
    if len(Top100)<100:
        Top100.append(TagDic[Tag])
    elif len(TagDic[Tag]['dates'])>len(Top100[0]['dates']):
        Top100[0]=TagDic[Tag]
    Top100 = sorted(Top100, key=lambda k: len(k['dates']))
for Val in Top100:
    dates = Val['dates']
    series = pd.Series([1]*len(dates), index=dates)
    TimeSeries = series.resample('D').sum()
    TSDic[Val['tag']] = TimeSeries

DfTS = pd.DataFrame(TSDic).fillna(0)

print(DfTS)

%matplotlib inline

Names = (".",".")
MinDist = sys.maxsize

for ColName1 in DfTS.columns:
    for ColName2 in DfTS.columns:
        if ColName1!=ColName2:
            Dist = EucDistance(DfTS[ColName1],DfTS[ColName2])
            if Dist<MinDist:
                MinDist = Dist
                Names = (ColName1,ColName2)
print(Names)  # These are the tags with the two closest time series
print(EucDistance(DfTS[Names[0]],DfTS[Names[1]])) # this is that closest distance


ts1 = DfTS[Names[0]]
ts2 = DfTS[Names[1]]
ts1.plot()
ts2.plot()


Choose a different distance/similarity metric and repeat the same time series analysis. Compare the two different metrics you used. **(10 pts)**

In [ ]:
%matplotlib inline

NamesL1 = (".",".")
MinDistL1 = sys.maxsize

for ColName1 in DfTS.columns:
    for ColName2 in DfTS.columns:
        if ColName1!=ColName2:
            Dist = L1Distance(DfTS[ColName1],DfTS[ColName2])
            if Dist<MinDistL1:
                MinDistL1 = Dist
                NamesL1 = (ColName1,ColName2)


print(NamesL1)  # These are the tags with the two closest time series using L1 dist
print(L1Distance(DfTS[NamesL1[0]],DfTS[NamesL1[1]])) # this is that closest L1 distance

ts3 = DfTS[NamesL1[0]]
ts4 = DfTS[NamesL1[1]]
ts3.plot()
ts4.plot()
